# Problem Set 3: Geometry
---
## Setup

In [134]:
# IPython magic
%load_ext autoreload
%autoreload 2
# Matplotlib magic
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [135]:
# Imports
import cv2
import matplotlib.pyplot as plt
import numpy as np
# Matplotlib params
plt.rcParams['figure.figsize'] = (14.0, 6.0)
plt.rcParams['axes.titlesize'] = 18
plt.rcParams['figure.titlesize'] = 22
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

---
## 1. Calibration

In [141]:
# Actual points in image A
with open('pts2d-pic_a.txt', 'r') as f:
    pts2d_a = np.array([[int(n) for n in line.split()] for line in f])
# Actual points in 3D space
with open('pts3d.txt', 'r') as f:
    pts3d = np.array([[float(n) for n in line.split()] for line in f])
# Normalized points in image A
with open('pts2d-norm-pic_a.txt', 'r') as f:
    pts2d_norm_a = np.array([[float(n) for n in line.split()] for line in f])
# Normalized points in 3D space
with open('pts3d-norm.txt', 'r') as f:
    pts3d_norm = np.array([[float(n) for n in line.split()] for line in f])

### 1.1 M from least squares

In [122]:
def projective_matrix_lsq(pts2d, pts3d):
    # Dims
    num_points = len(pts3d)
    num_variables = 11
    # AM = b
    A = np.zeros((2*num_points, num_variables), dtype=np.float)
    b = np.zeros((2*num_points), dtype=np.float)
    for ix, ([u, v], [x, y, z]) in enumerate(zip(pts2d, pts3d)):
        A[2*ix, :]   = [x, y, z, 1, 0, 0, 0, 0, -u*x, -u*y, -u*z]
        A[2*ix+1, :] = [0, 0, 0, 0, x, y, z, 1, -v*x, -v*y, -v*z]
        b[2*ix]   = u
        b[2*ix+1] = v
    # M = np.linalg.lstsq(A, b)[0]
    # M = (A'A)^-1.A'b
    M = np.linalg.inv(A.T@A) @ A.T @ b
    M = np.append(M, 1).reshape((3, 4))
    return M

In [125]:
M = projective_matrix_lsq(pts2d_norm_a, pts3d_norm)
print(M)

[[ 0.76785834 -0.49384797 -0.02339781  0.00674445]
 [-0.0852134  -0.09146818 -0.90652332 -0.08775678]
 [ 0.18265016  0.29882917 -0.07419242  1.        ]]


In [160]:
pt_3d = pts3d_norm[-1].copy()
print('Coordinates of point in 3D (x, y, z)   : ({:.4f}, {:7.4f}, {:7.4f})'.format(*pt_3d))
pt_3d = np.append(pt_3d, 1)
us, vs, s = M @ last_point
print('Calculated coordinates in Image (u, v) : ({:.4f}, {:7.4f})'.format(us/s, vs/s))
u, v = pts2d_norm_a[-1]
print('Actual coordinates in Image (u, v)     : ({:.4f}, {:7.4f})'.format(u, v))
resid = np.sqrt((u-us/s)**2 + (v-vs/s)**2)
print('Calculated residue                     : {:.4f}'.format(resid))

Coordinates of point in 3D (x, y, z)   : (1.2323,  1.4421,  0.4506)
Calculated coordinates in Image (u, v) : (242.2849, 165.8126)
Actual coordinates in Image (u, v)     : (0.1406, -0.4527)
Calculated residue                     : 293.7311


### 1.2 Number of points and M

In [137]:
# Points in image B
with open('pts2d-pic_b.txt', 'r') as f:
    pts2d_b = np.array([[int(n) for n in line.split()] for line in f])

In [149]:
sample_size = [8, 12, 16]
num_iters = 10
min_avg_resid = np.inf
for _ in range(num_iters):
    for k in sample_size:
        num_checks = 4
        sample = np.random.choice(num_points, k+num_checks, replace=False)
        pts2d_sample = pts2d_b[sample[:k]]
        pts3d_sample = pts3d[sample[:k]]
        M = projective_matrix_lsq(pts2d_sample, pts3d_sample)
        avg_resid = 0
        for ix in sample[k:]:
            pt3d = pts3d[ix].copy()
            pt3d = np.append(pt3d, 1)
            us, vs, s = M @ pt3d
            u, v = pts2d_b[ix]
            resid = np.sqrt((u-us/s)**2 + (v-vs/s)**2)
            avg_resid += resid / num_checks
        if avg_resid < min_avg_resid:
            min_avg_resid = avg_resid
            M_best = M
print(M)

[[-2.05008298e+00  1.19802723e+00  3.73792932e-01  2.41921658e+02]
 [-4.56220404e-01 -3.00593432e-01  2.15064770e+00  1.65159332e+02]
 [-2.24897072e-03 -1.09007895e-03  5.39849821e-04  1.00000000e+00]]


### 1.3 Center of camera

In [154]:
Q = M[:, :3]
m4 = M[:, 3]
C = -np.linalg.inv(Q) @ m4
print('Coordinates of camera (x, y, z): ({:.4f}, {:7.4f}, {:7.4f})'.format(*C))

Coordinates of camera (x, y, z): (303.0623, 307.1779, 30.4278)
